### ⚙️ Optimalizace výkonu modelů

Optimalizace výkonu je klíčová pro nasazení modelů v reálném čase, zejména na zařízeních s omezenými výpočetními zdroji (mobil, edge, embedded). Mezi hlavní techniky patří:

### 🧮 1. Kvantizace (Quantization)
Kvantizace převádí váhy a/nebo aktivace modelu z výchozího formátu (např. float32) na menší číselné reprezentace (int8, float16, bfloat16), čímž:

- snižuje velikost modelu,
- zrychluje inference,
- snižuje spotřebu paměti a energie.

**Typy kvantizace:**

- *Post-training*: Kvantizace po tréninku, bez nutnosti dalšího učení.
- *Quantization-aware training (QAT)*: Model se učí s kvantizačními efekty během tréninku.
- *Dynamic quantization*: Kvantizace vah, aktivace se kvantizují za běhu.
- *Static quantization*: Kvantizace vah i aktivací, vyžaduje kalibrační data.

- *float32*: 32 bitů - velmi přesné, ale extrémně náročné
- *float16*: rychlejší, středně přesné
- *bfloat16*: pro modely nejpoužívanější, středně přesné, exponent čísla je v FP32, umí grafiky s jádrem Pascal a vyšší (přibližně od roku 2016)
- *int8*: velmi rychlý, malý model, vhodné pro mobilní zařízení, edge ap.

### 🧪 Ukázkový kód: Post-training kvantizace v PyTorch

In [ ]:
import torch
from torchvision.models import resnet18

# 1. Načtení předtrénovaného modelu
model_fp32 = resnet18(pretrained=True)
model_fp32.eval()

# 2. Aplikace dynamické kvantizace
model_int8 = torch.quantization.quantize_dynamic(
    model_fp32,  # model
    {torch.nn.Linear},  # vrstvy k kvantizaci
    dtype=torch.qint8  # cílový formát
)

# 3. Uložení kvantizovaného modelu
torch.save(model_int8.state_dict(), "resnet18_int8.pth")

#### ✂️ 2. Pruning (prořezávání) - jen pro AI experty ...

Odstraňuje méně důležité váhy nebo celé kanály neuronové sítě.

*Unstructured pruning* – odstraňuje jednotlivé váhy.
*Structured pruning* – odstraňuje celé filtry nebo vrstvy.

In [ ]:
import torch.nn.utils.prune as prune

prune.l1_unstructured(model.layer1[0].conv1, name="weight", amount=0.3)

#### 📦 3. Batch inferencing

Zpracování více vstupů najednou (např. 8 obrázků místo 1) zvyšuje efektivitu inference na GPU.

- pokud máte dostatek vRAM/RAM tak dávky obrázků jsou nezbytnost!

In [ ]:
# vstupní batch: [batch_size, channels, height, width]
batch_input = torch.stack([img1, img2, img3, img4])
output = model(batch_input)

#### 🧠 Pokročilé kvantizační přístupy

Moderní frameworky (např. Intel Neural Compressor, NVIDIA TensorRT, OpenVINO) umožňují:

- per-layer kvantizaci – různé vrstvy mají různé formáty (např. int8 pro konvoluce, fp16 pro dense vrstvy),
- per-channel kvantizaci – každému kanálu je přiřazena vlastní kvantizační škála,
- mixed precision – kombinace fp16, bf16, int8 podle významu vrstvy.